In [92]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
import random
%matplotlib inline
import seaborn as sns
from sklearn.metrics import accuracy_score



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

import datetime as dt

In [93]:
train = pd.read_csv("../input/flight-delay-prediction-challenge/Train (8).csv")
test = pd.read_csv("../input/flight-delay-prediction-challenge/Test (9).csv")
sample = pd.read_csv('../input/flight-delay-prediction-challenge/SampleSubmission (5).csv')

In [94]:
train

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0
...,...,...,...,...,...,...,...,...,...,...
107828,train_id_107828,2018-07-05,WKL 0000,TUN,TUN,2018-07-05 23:00:00,2018-07-06 02.00.00,SCH,TU 32AIML,0.0
107829,train_id_107829,2018-01-13,UG 0003,DJE,TUN,2018-01-13 08:00:00,2018-01-13 09.00.00,SCH,UG AT7AT7,0.0
107830,train_id_107830,2018-11-07,SGT 0000,TUN,TUN,2018-11-07 05:00:00,2018-11-07 12.50.00,SCH,TU 736IOK,0.0
107831,train_id_107831,2018-01-23,UG 0010,TUN,DJE,2018-01-23 18:00:00,2018-01-23 18.45.00,ATA,TU CR9ISA,0.0


In [95]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [96]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
sample = reduce_mem_usage(sample)

Memory usage after optimization is: 7.61 MB
Decreased by 7.5%
Memory usage after optimization is: 0.64 MB
Decreased by 0.0%
Memory usage after optimization is: 0.09 MB
Decreased by 37.5%


In [97]:
train.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC', 'target'],
      dtype='object')

In [98]:
test.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC'],
      dtype='object')

In [99]:
train.drop("ID", axis=1, inplace=True)
test.drop("ID", axis=1, inplace=True)

In [100]:
gc.collect()

180

In [101]:

train["DATOP"]=pd.to_datetime(train["DATOP"])
train['Date_year'] =train["DATOP"].dt.year
train['Date_month'] = train["DATOP"].dt.month
train['Date_day'] = train["DATOP"].dt.day
train.drop("DATOP", axis=1, inplace=True)

test["DATOP"]=pd.to_datetime(test["DATOP"])
test['Date_year'] =test["DATOP"].dt.year
test['Date_month'] = test["DATOP"].dt.month
test['Date_day'] = test["DATOP"].dt.day
test.drop("DATOP", axis=1, inplace=True)

In [102]:

train["STD"]=pd.to_datetime(train["STD"])
train['STD_Date_year'] =train["STD"].dt.year
train['STD_Date_month'] = train["STD"].dt.month
train['STD_Date_day'] = train["STD"].dt.day
train['STD_Date_hour'] = train["STD"].dt.hour
train['STD_Date_minute'] =train["STD"].dt.minute
train['STD_Date_second'] = train["STD"].dt.second

train.drop("STD", axis=1, inplace=True)

test["STD"]=pd.to_datetime(test["STD"])
test['STD_Date_year'] =test["STD"].dt.year
test['STD_Date_month'] = test["STD"].dt.month
test['STD_Date_day'] = test["STD"].dt.day
test['STD_Date_hour'] = test["STD"].dt.hour
test['STD_Date_minute'] =test["STD"].dt.minute
test['STD_Date_second'] = test["STD"].dt.second

test.drop("STD", axis=1, inplace=True)

In [103]:
train["STAA"]=pd.to_datetime(train["STA"], format='%Y-%m-%d %H.%M.%S')
train['STA_Date_year'] =train["STAA"].dt.year
train['STA_Date_month'] = train["STAA"].dt.month
train['STA_Date_day'] = train["STAA"].dt.day
train['STA_Date_hour'] = train["STAA"].dt.hour
train['STA_Date_minute'] =train["STAA"].dt.minute
train['STA_Date_second'] = train["STAA"].dt.second
train.drop("STA", axis=1, inplace=True)
train.drop("STAA", axis=1, inplace=True)

test["STAA"]=pd.to_datetime(test["STA"], format='%Y-%m-%d %H.%M.%S')
test['STA_Date_year'] =test["STAA"].dt.year
test['STA_Date_month'] = test["STAA"].dt.month
test['STA_Date_day'] = test["STAA"].dt.day
test['STA_Date_hour'] = test["STAA"].dt.hour
test['STA_Date_minute'] =test["STAA"].dt.minute
test['STA_Date_second'] = test["STAA"].dt.second
test.drop("STA", axis=1, inplace=True)
test.drop("STAA", axis=1, inplace=True)

In [104]:
train_df = train.drop("target", axis=1)

In [105]:
from category_encoders import CountEncoder
enc = CountEncoder(normalize=True, cols=['DEPSTN', 'ARRSTN','STATUS','AC',"FLTID"])
train_df = enc.fit_transform(train_df)
test=enc.transform(test)

In [106]:
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# X_train = mms.fit_transform(train_df)
# X_test = mms.transform(test)

In [107]:
# X_train.shape

(107833, 20)

In [108]:
col = train_df.columns

In [109]:
col

Index(['FLTID', 'DEPSTN', 'ARRSTN', 'STATUS', 'AC', 'Date_year', 'Date_month',
       'Date_day', 'STD_Date_year', 'STD_Date_month', 'STD_Date_day',
       'STD_Date_hour', 'STD_Date_minute', 'STD_Date_second', 'STA_Date_year',
       'STA_Date_month', 'STA_Date_day', 'STA_Date_hour', 'STA_Date_minute',
       'STA_Date_second'],
      dtype='object')

In [116]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

models = []
models.append(LGBMRegressor(random_state=17, max_depth = 3))
models.append(LGBMRegressor(random_state=17,  max_depth = 5))
models.append(XGBRegressor(random_state=17,  max_depth = 3))
models.append(XGBRegressor(random_state=17, max_depth = 5))
models.append(CatBoostRegressor(random_state=17,  max_depth=3))
models.append(CatBoostRegressor(random_state=17,  max_depth=5))

In [117]:
%%time
for i in range(len(models)):
  models[i].fit(train_df,train['target'])

Learning rate set to 0.085774
0:	learn: 116.6994089	total: 27.6ms	remaining: 27.6s
1:	learn: 116.3450878	total: 42.4ms	remaining: 21.2s
2:	learn: 116.0326953	total: 58.1ms	remaining: 19.3s
3:	learn: 115.7709561	total: 75.1ms	remaining: 18.7s
4:	learn: 115.5556908	total: 91.3ms	remaining: 18.2s
5:	learn: 115.3749820	total: 107ms	remaining: 17.7s
6:	learn: 115.2068296	total: 122ms	remaining: 17.3s
7:	learn: 115.0676622	total: 137ms	remaining: 17s
8:	learn: 114.9466105	total: 152ms	remaining: 16.7s
9:	learn: 114.8435486	total: 166ms	remaining: 16.4s
10:	learn: 114.7583254	total: 182ms	remaining: 16.3s
11:	learn: 114.6770350	total: 197ms	remaining: 16.3s
12:	learn: 114.6045271	total: 211ms	remaining: 16s
13:	learn: 114.5243452	total: 224ms	remaining: 15.8s
14:	learn: 114.4642069	total: 238ms	remaining: 15.6s
15:	learn: 114.4069188	total: 252ms	remaining: 15.5s
16:	learn: 114.3503833	total: 266ms	remaining: 15.4s
17:	learn: 114.2872716	total: 280ms	remaining: 15.3s
18:	learn: 114.2447516	to

In [119]:
predicts = []
for i in range(len(models)):
  predicts.append(models[i].predict(test[col]))

In [121]:
test['target'] = np.mean(predicts, axis=0)

In [123]:
sample.target = test.target

In [124]:
sample

,ID,target
0,test_id_0,14.165223
1,test_id_1,65.748571
2,test_id_2,33.152577
3,test_id_3,24.075797
4,test_id_4,26.917518
...,...,...
9328,test_id_9328,112.576671
9329,test_id_9329,25.459203
9330,test_id_9330,34.412402
9331,test_id_9331,15.140364


In [125]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")

In [126]:
create_submission(sample, "sub5")

/kaggle/working/sub5.csv